# Introduction

## Présentation

Redis est une base de données NoSQL qui se classe dans la catégorie "data structure server". C'est  un système permettant de traiter, de manière très performante, des données sous la forme clés/valeurs. Cette performance est obtenue car les données sont stockées directement sur la mémoire RAM. Cela permet d'accéder très facilement et très rapidement aux données 

La clé permet de récupérer et d'identifier de manière unique la donnée. Redis ne permet pas de stocker des modèles de données très complexes, il faut donc ruser pour créer une structure de données complète et complexe.

Redis est souvent utilisé dans le developpement web pour gérer les sessions utilisateurs, leur panier, leurs configurations. En outre, lorsqu'on stocke une petite quantité de donnée et qu'on a besoin d'y accèder en quelques milliseconds

## Les données 



Les valeurs peuvent appartenir aux types suivants :

- chaînes de caractères
- compteurs numériques (les données numériques seront stockées dans la base sous la forme de chaîne de caractères)
- listes 
- sets ou sets ordonnés
- dictionnaires

## Lancer un container Redis

Comme pour Mongo, il nous faut une instance redis qui tourne en local, pour se faire il suffit de lancer un container Redis

Vous devriez être familier avec Docker maintenant, à vous de jouer pour lancer ce container !

## Installation

Le package Redis pour python est déjà installé, voici la commande si jamais :

`pipenv install redis==5.0.8`

# Let's play

In [2]:
import redis

In [3]:
redis_client = redis.StrictRedis(host='localhost')

Pour tester la connexion on peut pinguer la base de données :

In [4]:
# First, run this command in your terminal to start Redis:
# docker run -d -p 6379:6379 redis:latest

# Then test the connection
redis_client.ping()

True

Vous pouvez regarder toutes les configurations utilisées par Redis, beaucoup ne sont pas utiles à notre niveau, mais vous pouvez voir par exemple, `database`, `maxclients`, `port` , `timeout` qui peuvent être utile de modifier pour différentes applications

In [5]:
redis_client.config_get()

{'cluster-announce-ip': '',
 'sanitize-dump-payload': 'no',
 'shutdown-on-sigterm': 'default',
 'appendfilename': 'appendonly.aof',
 'search-default-scorer': 'BM25STD',
 'cluster-slot-migration-write-pause-timeout': '10000',
 'ts-retention-policy': '0',
 'notify-keyspace-events': '',
 'repl-timeout': '60',
 'search-friso-ini': '',
 'cluster-node-timeout': '15000',
 'search-_prioritize-intersect-union-children': 'no',
 'slave-ignore-maxmemory': 'yes',
 'hash-max-ziplist-value': '64',
 'cf-max-iterations': '20',
 'bf-error-rate': '0.01',
 'acllog-max-len': '128',
 'syslog-enabled': 'no',
 'cf-bucket-size': '2',
 'lfu-log-factor': '10',
 'cluster-link-sendbuf-limit': '0',
 'ts-encoding': 'compressed',
 'hz': '10',
 'slaveof': '',
 'replica-serve-stale-data': 'yes',
 'acl-pubsub-default': 'resetchannels',
 'tls-port': '0',
 'tcp-keepalive': '300',
 'bind-source-addr': '',
 'aclfile': '',
 'search-_print-profile-clock': 'yes',
 'oom-score-adj': 'no',
 'dir': '/data',
 'enable-debug-command'

## Stockage clé valeurs

Pour stocker une valeur correspondant à une clé 

In [6]:
KEY = "name"
VALUE = "yourname"
redis_client.set(KEY, VALUE)

True

True montre que le stockage s'est bien déroulé. 

Pour récupérer la valeur stockée il suffit d'appeler la méthode `GET` sur cette clé

In [7]:
redis_client.get(KEY)

b'yourname'

Vous pouvez voir qu'à l'aide du `b` que la réponse n'est pas exactement ce que l'on a stocké, mais une représentation en bytes de notre chaine de charactères. 

In [8]:
assert redis_client.get(KEY) == VALUE, "Les valeurs ne correspondent pas"

AssertionError: Les valeurs ne correspondent pas

Pour cela, en python, il faut appeler la méthode `decode()` pour transformer des bytes en string 

In [9]:
assert redis_client.get(KEY).decode() == VALUE, "les valeurs ne correspondent pas"

On peut préciser au client de décoder les réponses et de les transformer directement en strings

In [10]:
redis_client = redis.StrictRedis(host='localhost', charset='utf-8', decode_responses=True)

In [11]:
assert redis_client.get(KEY) == VALUE

Les valeurs peuvent aussi être des entiers

In [12]:
KEY = "age"
VALUE = 20

redis_client.set(KEY, VALUE)

True

In [13]:
redis_client.get(KEY)

'20'

Vous pouvez alors utiliser la méthode `incr` pour incrémenter cet entier. Il faut faire attention puisque redis renvoie toujours les entiers comme des chaines de charactères.

In [14]:
redis_client.incr(KEY)

21

## Exemple 

In [15]:
%pip install ipywidgets
from ipywidgets import widgets, interact
from IPython.display import display

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
KEY = "click_number"
DEFAULT_VALUE = 0

redis_client.set(KEY, DEFAULT_VALUE)

button = widgets.Button(description="Click me!")
def on_button_clicked(button):
    redis_client.incr(KEY)
    print(f"Le bouton à été cliqué {redis_client.get(KEY)} fois")
    button.description = "Click me more !"
    
button.on_click(on_button_clicked)
display(button)

Button(description='Click me!', style=ButtonStyle())

In [18]:
print(f"Tu as cliqué sur le bouton {redis_client.get(KEY)} fois !")

Tu as cliqué sur le bouton 11 fois !


## Les Listes

Redis permet aussi de stocker des valeurs de type `list`, pour cela nous avons à notre disposition un ensemble de méthodes :  

- La méthode `LPUSH`  permet d'ajouter un élément en tête de list la complexité est de O(1).  
- La méthode `RPUSH` permet d'ajouter un élément en queue de list la complexité est de O(1).  
- La méthode `LINSERT` permet d'ajouter un élément avant ou après la première occurence d'une certaine valeur, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `LSET` permet de modifier la valeur correspondant à l'index spécifié, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `RPOP` permet de supprimer un élément en queue de list la complexité est de O(1).
- La méthode `LPOP` permet de supprimer un élément en tête de list la complexité est de O(1).
- La méthode `LINDEX` permet d'afficher un élément en fonction de son index.
- La méthode `LREM` permet de supprimer un élément en fonction de son index.
- La méthode `LLEN` permet de récupérer la taille d'une liste.

Nous allons toutes les tester.

In [19]:
redis_client.lpush('language_list', "kotlin")
redis_client.lpush('language_list', "python")

2

On peut aussi ajouter plusieurs éléments en même temps dans une liste

In [20]:
redis_client.lpush('language_list', "java", "c++")

4

In [21]:
f"La taille de la liste est {redis_client.llen('language_list')}"

'La taille de la liste est 4'

In [22]:
INDEX = 2
f"L'élément à l'index {INDEX} de la liste est '{redis_client.lindex('language_list', INDEX)}'"

"L'élément à l'index 2 de la liste est 'python'"

Le client renvoie la taille de la liste venant d'être mise à jour. On peut afficher tous les éléments de la liste en les retirant un par un.

In [23]:
while(redis_client.llen('language_list')!=0):

    print(redis_client.lpop('language_list'))

c++
java
python
kotlin


## Les dictionnaires 

Redis permet de stocker assez facilement des dictionnaires, on peut comme cela stocker des structures de données plus complexes. Pour cela, nous avons plusieurs méthodes : 

- La méthode `HMSET` permet d'ajouter un dictionnaire à une clé
- La méthode `HSET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGETALL` permet de récupérer toutes les clés valeurs d'un dictionnaire.
- La méthode `HKEYS` permet de récupérer toutes les clés d'un dictionnaire
- La méthode `HVALS` permet de récupérer toutes les valeurs d'un dictionnaire

In [24]:
doc = {
    "name":"yourname",
    "age":20,
    "city":"paris"
}

In [25]:
redis_client.hset("user1", mapping=doc)

3

In [26]:
redis_client.hgetall("user1")

{'name': 'yourname', 'age': '20', 'city': 'paris'}

On peut modifier uniquement une clé du dictionnaire

In [27]:
redis_client.hset("user1", "city", "lyon")

0

In [28]:
redis_client.hgetall("user1")

{'name': 'yourname', 'age': '20', 'city': 'lyon'}

## Les pipelines

In [29]:
import timeit

Lorsque l'on appelle une méthode depuis le client python, chaqu'une d'entre elles fait un appel à Redis, ce qui peut prendre du temps. Les pipelines permettent de réduire ce temps en stockant toutes les opérations et en les applicant une seule fois sur la base. Cela permet d'augmenter grandement les performances en réduisant les appels et le passage par le réseau. 

In [30]:
N = 10000

In [31]:
def store_key_values(N):
    for i in range(N):
        redis_client.set(f"key_{i}", f"value_{i}")

In [32]:
%%timeit
store_key_values(N=N)

6.55 s ± 910 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
def store_key_values_with_pipelines(N):
    p = redis_client.pipeline()
    for i in range(N):
        p.set(f"key_{i}", f"value_{i}")

In [34]:
%%timeit
store_key_values_with_pipelines(N=N)

12.4 ms ± 214 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


On voit alors que l'on peut diviser le temps par 100. Dans vos différents métiers les performances sont souvent très importantes

# Exercice

Utiliser redis pour gérer les commandes d'un restaurant de burgers, pour simplifier les choses les clients pourront choisir leurs ingrédients dans 3 types de produits :

- Pain: blanc, complet, aux céréales
- Garniture : salade, tomates , oignons, concombres
- Protéines : poulet, boeuf, agneau, tofu

Chaque ingrédient équivaut à un prix et un poids, avec chaque commande, vous devrez calculer le poids et le prix que les clients devront régler. 

1) Stocker les informations concernant les ingrédients dans redis   
2) Créer la méthode permettant de créer aléatoirement un burger  
3) Créer la méthode qui permet d'empiler dans redis les prochaines commandes (Réfléchir au type de données utilisé)  
4) Créer la méthode permettant de calculer le poids et le prix du burger  
5) Créer une boucle pour dépiler, réaliser et servir les burgers.  

Pour vous aider voici quelques données 

In [35]:
pains = {
    "blanc":{
        "poid":100,
        "prix": 2
    },
    "complet":{
        "poid":90,
        "prix": 3
    },
    "cereales":{
        "poid":120,
        "prix": 4
    }
}

In [36]:
garniture = {
    "salade":{
        "poid":50,
        "prix": 1
    }, "tomates":{
        "poid":60,
        "prix": 2
    } , "oignons":{
        "poid":10,
        "prix": 1.5
    }, "concombres":{
        "poid":30,
        "prix": 3
    }
}

In [37]:
proteine = {
    "poulet":{
        "poid":100,
        "prix": 6
    }, "boeuf":{
        "poid":150,
        "prix": 5
    } , "agneau":{
        "poid":120,
        "prix": 7
    }, "tofu":{
        "poid":80,
        "prix": 8
    }
}

In [39]:
import random
import json

# 1) Stocker les informations des ingrédients dans Redis
def store_ingredients_in_redis():
    """Stocke tous les ingrédients avec leur prix et poids dans Redis sous forme de hash"""
    # Stocker les pains
    for pain_name, pain_data in pains.items():
        redis_client.hset(f"pain:{pain_name}", mapping=pain_data)
    
    # Stocker les garnitures
    for garniture_name, garniture_data in garniture.items():
        redis_client.hset(f"garniture:{garniture_name}", mapping=garniture_data)
    
    # Stocker les protéines
    for proteine_name, proteine_data in proteine.items():
        redis_client.hset(f"proteine:{proteine_name}", mapping=proteine_data)
    
    print("✓ Ingrédients stockés dans Redis")

# 2) Créer aléatoirement un burger
def make_burger():
    """Crée un burger aléatoire en choisissant un pain, une garniture et une protéine"""
    pain_choisi = random.choice(list(pains.keys()))
    garniture_choisie = random.choice(list(garniture.keys()))
    proteine_choisie = random.choice(list(proteine.keys()))
    
    burger = {
        "pain": pain_choisi,
        "garniture": garniture_choisie,
        "proteine": proteine_choisie
    }
    
    return burger

# 3) Empiler une commande dans Redis (utilise une liste comme file FIFO)
def make_order():
    """Crée un burger et l'ajoute à la file d'attente des commandes"""
    burger = make_burger()
    # Utiliser RPUSH pour ajouter en queue (FIFO avec LPOP)
    redis_client.rpush("orders_queue", json.dumps(burger))
    print(f"✓ Commande ajoutée: Pain {burger['pain']}, {burger['garniture']}, {burger['proteine']}")
    return burger

# 4) Récupérer la prochaine commande
def get_next_order():
    """Dépile la prochaine commande de la file d'attente"""
    order_json = redis_client.lpop("orders_queue")
    if order_json:
        return json.loads(order_json)
    return None

# 5) Calculer le prix et le poids d'un burger
def process_price_weight(burger):
    """Calcule le prix total et le poids total d'un burger"""
    # Récupérer les données depuis Redis
    pain_data = redis_client.hgetall(f"pain:{burger['pain']}")
    garniture_data = redis_client.hgetall(f"garniture:{burger['garniture']}")
    proteine_data = redis_client.hgetall(f"proteine:{burger['proteine']}")
    
    # Calculer prix et poids totaux
    prix_total = float(pain_data['prix']) + float(garniture_data['prix']) + float(proteine_data['prix'])
    poids_total = int(pain_data['poid']) + int(garniture_data['poid']) + int(proteine_data['poid'])
    
    return prix_total, poids_total

# 6) Traiter un burger complet
def process_burger():
    """Récupère une commande, calcule son prix et poids, et retourne les infos"""
    burger = get_next_order()
    
    if burger is None:
        return None, None, None
    
    prix, poids = process_price_weight(burger)
    burger_description = f"Pain {burger['pain']}, {burger['garniture']}, {burger['proteine']}"
    
    return burger_description, prix, poids

In [40]:
# Initialisation: stocker les ingrédients dans Redis
store_ingredients_in_redis()

# Créer plusieurs commandes aléatoires
print("\n=== CRÉATION DES COMMANDES ===")
nb_commandes = 5
for i in range(nb_commandes):
    make_order()

# Traiter toutes les commandes
print(f"\n=== TRAITEMENT DES {nb_commandes} COMMANDES ===")
while True:
    burger, price, weight = process_burger()
    
    if burger is None:
        print("\n✓ Toutes les commandes ont été traitées!")
        break
    
    print(f"Le burger [{burger}] de {weight}g a été préparé et servi - Prix: {price}€")

✓ Ingrédients stockés dans Redis

=== CRÉATION DES COMMANDES ===
✓ Commande ajoutée: Pain blanc, concombres, tofu
✓ Commande ajoutée: Pain cereales, salade, boeuf
✓ Commande ajoutée: Pain blanc, oignons, agneau
✓ Commande ajoutée: Pain blanc, oignons, boeuf
✓ Commande ajoutée: Pain complet, salade, poulet

=== TRAITEMENT DES 5 COMMANDES ===
Le burger [Pain blanc, concombres, tofu] de 210g a été préparé et servi - Prix: 13.0€
Le burger [Pain cereales, salade, boeuf] de 320g a été préparé et servi - Prix: 10.0€
Le burger [Pain blanc, oignons, agneau] de 230g a été préparé et servi - Prix: 10.5€
Le burger [Pain blanc, oignons, boeuf] de 260g a été préparé et servi - Prix: 8.5€
Le burger [Pain complet, salade, poulet] de 240g a été préparé et servi - Prix: 10.0€

✓ Toutes les commandes ont été traitées!
